# Day 9: Sensor Boost
You've just said goodbye to the rebooted rover and left Mars when you receive a faint distress signal coming from the asteroid belt. It must be the Ceres monitoring station!

In order to lock on to the signal, you'll need to boost your sensors. The Elves send up the latest BOOST program - Basic Operation Of System Test.

While BOOST (your puzzle input) is capable of boosting your sensors, for tenuous safety reasons, it refuses to do so until the computer it runs on passes some checks to demonstrate it is a complete Intcode computer.

Your existing Intcode computer is missing one key feature: it needs support for parameters in relative mode.

Parameters in mode 2, relative mode, behave very similarly to parameters in position mode: the parameter is interpreted as a position. Like position mode, parameters in relative mode can be read from or written to.

The important difference is that relative mode parameters don't count from address 0. Instead, they count from a value called the relative base. The relative base starts at 0.

The address a relative mode parameter refers to is itself plus the current relative base. When the relative base is 0, relative mode parameters and position mode parameters with the same value refer to the same address.

For example, given a relative base of 50, a relative mode parameter of -7 refers to memory address 50 + -7 = 43.

The relative base is modified with the relative base offset instruction:

Opcode 9 adjusts the relative base by the value of its only parameter. The relative base increases (or decreases, if the value is negative) by the value of the parameter.
For example, if the relative base is 2000, then after the instruction 109,19, the relative base would be 2019. If the next instruction were 204,-34, then the value at address 1985 would be output.

Your Intcode computer will also need a few other capabilities:

- The computer's available memory should be much larger than the initial program. Memory beyond the initial program starts with the value 0 and can be read or written like any other memory. (It is invalid to try to access memory at a negative address, though.)
- The computer should have support for large numbers. Some instructions near the beginning of the BOOST program will verify this capability.
Here are some example programs that use these features:

```
109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99 takes no input and produces a copy of itself as output.
1102,34915192,34915192,7,4,7,99,0 should output a 16-digit number.
104,1125899906842624,99 should output the large number in the middle.
```

The BOOST program will ask for a single input; run it in test mode by providing it the value 1. It will perform a series of checks on each opcode, output any opcodes (and the associated parameter modes) that seem to be functioning incorrectly, and finally output a BOOST keycode.

Once your Intcode computer is fully functional, the BOOST program should report no malfunctioning opcodes when run in test mode; it should only output a single value, the BOOST keycode. What BOOST keycode does it produce?

In [2]:
from intcode import *

In [81]:
_ALL_OPS = []
YIELD = object()

class StdioEnv:
    outputs = ()

    def reset(self):
        pass

    def input(self):
        return int(input('Requesting program input> '))

    def output(self, value):
        print("Output:", value)


class ProgrammaticEnv:
    def __init__(self, inputs):
        self.inputs = list(inputs)
        self.reset()

    def reset(self):
        self._iter = iter(self.inputs)
        self.outputs = []

    def input(self):
        return int(next(self._iter))

    def output(self, value):
        self.outputs.append(value)
        
    def __repr__(self):
        return f"ProgrammaticEnv(outputs={self.outputs})"
    
    
class Tape:
    def __init__(self, data):
        self.data = data
        
    def __getitem__(self, position):
        if isinstance(position, slice):
            # This should allow enforcing the below logic for slicing, which is
            # mainly only used to get interpreter arguments; it might be important
            # that for instance an instruction is inferring data from a "larger" tape
            self.ensure_size(max(position.start, position.stop))
            return self.data[position]
        
        if position < 0:
            raise Exception("Negative tape index")
        elif position >= len(self.data):
            return 0
        else:
            return self.data[position]
        
    def __setitem__(self, position, value):
        if position < 0:
            raise Exception("Setting negative tape index")
        self.ensure_size(position)
        self.data[position] = value
        
    def ensure_size(self, size):
        if size >= len(self.data):
            self.data += [0] * (1 + size - len(self.data))
        
    def __repr__(self):
        return f'Tape{self.data}'

    
class CallContext:
    def __init__(self, interpreter_ctx, argspec, op, call_mode, inputs):
        self.interpreter_ctx = interpreter_ctx
        self.tape = interpreter_ctx.tape
        self.env = interpreter_ctx.env
        self.mode = call_mode
        self.inputs = inputs
        self.argspec = argspec
        self.op = op
        
    def __getitem__(self, index):
        argspec = self.argspec[index]
        literal = self.inputs[index]
        if argspec is LITERAL:
            return literal
        elif argspec is ADDRESS:
            return self.resolve_address(index)
        else:
            return self.resolve_position(index)
        
    def resolve_address(self, index):
        mode = self.mode[index]
        literal = self.inputs[index]
        if mode == 2:
            return self.interpreter_ctx.relative_base + literal
        else:
            return literal
        
    def resolve_position(self, index):
        mode = self.mode[index]
        if mode == 1:
            return self.inputs[index]
        else:
            return self.tape[self.resolve_address(index)]
    
    def __len__(self):
        return len(self.inputs)
    
    def arg_repr(self, index):
        argspec = self.argspec[index]
        literal = self.inputs[index]
        mode = self.mode[index]
        if argspec is LITERAL:
            return f'{literal}L'
        elif argspec is ADDRESS:
            if mode == 2:
                position = literal + self.interpreter_ctx.relative_base
                return f'[{position} = {literal}R{self.interpreter_ctx.relative_base}](->{self.tape[position]})'
            else:
                position = literal
                return f'[{position}](->{self.tape[position]})'
        else:
            mode = self.mode[index]
            if mode == 2:
                position = literal + self.interpreter_ctx.relative_base
                return f'[{position} = {literal}R{self.interpreter_ctx.relative_base}](->{self.tape[position]})'
            elif mode == 1:
                return f'{literal}L'
            else:
                position = literal
                return f'[{position}](->{self.tape[position]})'
    
    def __repr__(self):
        return f'CallContext: {self.op.__name__} {" ".join(self.arg_repr(i) for i in range(len(self)))}'
    

class InterpreterContext:
    def __init__(self, tape, env, index=0, state=None, trampoline=False, debug=False):
        self.tape = Tape(tape)
        self.env = env
        self.state = state
        self.debug = debug
        self.trampoline = trampoline
        self.index = index
        self.relative_base = 0
        
    def __repr__(self):
        return f'''
            InterpreterContext(
                index={self.index},
                state={self.state},
                relative_base={self.relative_base},
                tape={self.tape},
            )
            '''

class IntcodeInterpreter:
    def __init__(self):
        self.registered_ops = {}
        self.register_default_ops(_ALL_OPS)
        
    def evaluate(self, tape, env=StdioEnv(), trampoline=False, debug=False):
        tape = list(tape)
        ctx = InterpreterContext(tape, env, trampoline=trampoline, debug=debug)
        ctx.env.reset()
        return self.evaluate_from_context(ctx)
    
    def evaluate_from_context(self, ctx):
        ctx.state = 'RUNNING'
        tape = ctx.tape
        debug = ctx.debug
        index = ctx.index
        while tape[index] != HALT:
            opcode = tape[index]
            opcode_base = opcode % 100
            op, argspec = self.registered_ops[opcode_base]
            argcount = len(argspec)
            mode_v = (opcode - (opcode % 100)) // 100
            mode_s = str(mode_v)
            padded_mode_s = '0' * (argcount - len(mode_s)) + mode_s
            mode = [int(c) for c in reversed(padded_mode_s)]
            next_index = index + argcount + 1
            call_ctx = CallContext(ctx, argspec, op, mode, tape[index+1:next_index])
            if debug:
                print(call_ctx)
            ret = op(call_ctx)
            index = next_index if ret is None else ret
            ctx.index = index
            
            # Handle trampoline YIELD cases
            if op is output and ctx.trampoline:
                ctx.state = YIELD
                return ctx
        
        ctx.state = HALT
        return ctx
    
    def register_default_ops(self, all_ops):
        for fn, opcode, argspec, is_official in all_ops:
            if is_official:
                self.registered_ops[opcode] = (fn, argspec)
                
    
def register_op(opcode: int, args, is_official=True):
    def decorator(fn):
        _ALL_OPS.append((fn, opcode, args, is_official))
        return fn
    return decorator

LITERAL, ADDRESS, L_OR_A = object(), object(), object()

@register_op(opcode=1, args=[L_OR_A, L_OR_A, ADDRESS])
def add(ctx):
    a, b, oi = ctx
    ctx.tape[oi] = a + b

@register_op(opcode=2, args=[L_OR_A, L_OR_A, ADDRESS])
def multiply(ctx):
    a, b, oi = ctx
    ctx.tape[oi] = a * b

@register_op(opcode=3, args=[ADDRESS])
def _input(ctx):
    (oi,) = ctx
    ctx.tape[oi] = int(ctx.env.input())

@register_op(opcode=4, args=[L_OR_A])
def output(ctx):
    (v,) = ctx
    ctx.env.output(v)

@register_op(opcode=5, args=[L_OR_A, L_OR_A])
def jump_if_true(ctx):
    v, ji = ctx
    if v:
        return ji

@register_op(opcode=6, args=[L_OR_A, L_OR_A])
def jump_if_false(ctx):
    v, ji = ctx
    if not v:
        return ji

@register_op(opcode=7, args=[L_OR_A, L_OR_A, ADDRESS])
def less_than(ctx):
    a, b, oi = ctx
    ctx.tape[oi] = int(a < b)

@register_op(opcode=8, args=[L_OR_A, L_OR_A, ADDRESS])
def equals(ctx):
    a, b, oi = ctx
    ctx.tape[oi] = int(a == b)
    
@register_op(opcode=9, args=[L_OR_A])
def set_relative_base(ctx):
    (base_update,) = ctx
    ctx.interpreter_ctx.relative_base += base_update

In [82]:
interpreter = IntcodeInterpreter()

In [84]:
interpreter.evaluate(make_tape('109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99'))

Output: 109
Output: 1
Output: 204
Output: -1
Output: 1001
Output: 100
Output: 1
Output: 100
Output: 1008
Output: 100
Output: 16
Output: 101
Output: 1006
Output: 101
Output: 0
Output: 99



            InterpreterContext(
                index=15,
                state=99,
                relative_base=16,
                tape=Tape[109, 1, 204, -1, 1001, 100, 1, 100, 1008, 100, 16, 101, 1006, 101, 0, 99, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 1],
            )
            

In [85]:
interpreter.evaluate(make_tape('1102,34915192,34915192,7,4,7,99,0'))

Output: 1219070632396864



            InterpreterContext(
                index=6,
                state=99,
                relative_base=0,
                tape=Tape[1102, 34915192, 34915192, 7, 4, 7, 99, 1219070632396864],
            )
            

In [86]:
interpreter.evaluate(make_tape('104,1125899906842624,99'))

Output: 1125899906842624



            InterpreterContext(
                index=2,
                state=99,
                relative_base=0,
                tape=Tape[104, 1125899906842624, 99],
            )
            

In [57]:
import tools

In [58]:
data = tools.get_data(9)

In [87]:
interpreter.evaluate(make_tape(data), debug=True)

CallContext: multiply 34463338L 34463338L [63](->0)
CallContext: less_than [63](->1187721666102244) 34463338L [63](->1187721666102244)
CallContext: jump_if_true [63](->0) 53L
CallContext: multiply 1L 3L [1000](->0)
CallContext: set_relative_base 988L
CallContext: set_relative_base [1000 = 12R988](->3)
CallContext: set_relative_base [1000](->3)
CallContext: set_relative_base [1000 = 6R994](->3)
CallContext: set_relative_base [1000 = 3R997](->3)
CallContext: _input [1000 = 0R1000](->3)
Requesting program input> 1
CallContext: equals [1000](->1) 1L [63](->0)
CallContext: jump_if_true [63](->1) 65L
CallContext: add 0L 0L [1020](->0)
CallContext: multiply 1L 800L [1023](->0)
CallContext: add 0L 388L [1025](->0)
CallContext: add 0L 31L [1012](->0)
CallContext: multiply 1L 1L [1021](->0)
CallContext: add 22L 0L [1014](->0)
CallContext: add 0L 30L [1002](->0)
CallContext: add 0L 716L [1027](->0)
CallContext: multiply 32L 1L [1009](->0)
CallContext: add 0L 38L [1017](->0)
CallContext: multiply 


            InterpreterContext(
                index=903,
                state=99,
                relative_base=1014,
                tape=Tape[1102, 34463338, 34463338, 63, 1007, 63, 34463338, 63, 1005, 63, 53, 1102, 1, 3, 1000, 109, 988, 209, 12, 9, 1000, 209, 6, 209, 3, 203, 0, 1008, 1000, 1, 63, 1005, 63, 65, 1008, 1000, 2, 63, 1005, 63, 904, 1008, 1000, 0, 63, 1005, 63, 58, 4, 25, 104, 0, 99, 4, 0, 104, 0, 99, 4, 17, 104, 0, 99, 1, 2436480432, 1101, 0, 0, 1020, 1102, 1, 800, 1023, 1101, 0, 388, 1025, 1101, 0, 31, 1012, 1102, 1, 1, 1021, 1101, 22, 0, 1014, 1101, 0, 30, 1002, 1101, 0, 716, 1027, 1102, 32, 1, 1009, 1101, 0, 38, 1017, 1102, 20, 1, 1015, 1101, 33, 0, 1016, 1101, 0, 35, 1007, 1101, 0, 25, 1005, 1102, 28, 1, 1011, 1102, 1, 36, 1008, 1101, 0, 39, 1001, 1102, 1, 21, 1006, 1101, 397, 0, 1024, 1102, 1, 807, 1022, 1101, 0, 348, 1029, 1101, 0, 23, 1003, 1101, 29, 0, 1004, 1102, 1, 26, 1013, 1102, 34, 1, 1018, 1102, 1, 37, 1010, 1101, 0, 27, 1019, 1102, 24, 1, 1000, 1101, 3

### Part Two
You now have a complete Intcode computer.

Finally, you can lock on to the Ceres distress signal! You just need to boost your sensors using the BOOST program.

The program runs in sensor boost mode by providing the input instruction the value 2. Once run, it will boost the sensors automatically, but it might take a few seconds to complete the operation on slower hardware. In sensor boost mode, the program will output a single value: the coordinates of the distress signal.

Run the BOOST program in sensor boost mode. What are the coordinates of the distress signal?

In [88]:
interpreter.evaluate(make_tape(data))

Requesting program input> 2
Output: 45710



            InterpreterContext(
                index=921,
                state=99,
                relative_base=1000,
                tape=Tape[1102, 34463338, 34463338, 63, 1007, 63, 34463338, 63, 1005, 63, 53, 1102, 1, 3, 1000, 109, 988, 209, 12, 9, 1000, 209, 6, 209, 3, 203, 0, 1008, 1000, 1, 63, 1005, 63, 65, 1008, 1000, 2, 63, 1005, 63, 904, 1008, 1000, 0, 63, 1005, 63, 58, 4, 25, 104, 0, 99, 4, 0, 104, 0, 99, 4, 17, 104, 0, 99, 1, 0, 1101, 0, 0, 1020, 1102, 1, 800, 1023, 1101, 0, 388, 1025, 1101, 0, 31, 1012, 1102, 1, 1, 1021, 1101, 22, 0, 1014, 1101, 0, 30, 1002, 1101, 0, 716, 1027, 1102, 32, 1, 1009, 1101, 0, 38, 1017, 1102, 20, 1, 1015, 1101, 33, 0, 1016, 1101, 0, 35, 1007, 1101, 0, 25, 1005, 1102, 28, 1, 1011, 1102, 1, 36, 1008, 1101, 0, 39, 1001, 1102, 1, 21, 1006, 1101, 397, 0, 1024, 1102, 1, 807, 1022, 1101, 0, 348, 1029, 1101, 0, 23, 1003, 1101, 29, 0, 1004, 1102, 1, 26, 1013, 1102, 34, 1, 1018, 1102, 1, 37, 1010, 1101, 0, 27, 1019, 1102, 24, 1, 1000, 1101, 353, 0, 10